In [1]:
import sys

from data_helper import ImageGenerator, get_train_matrices
from sklearn.cross_validation import train_test_split
from keras_helper import KerasModel

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Using TensorFlow backend.
/home/westside/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
img_size = (128, 128)
img_channels = 3
output_size = 17

In [3]:
x_train, y_train = get_train_matrices("../data/train_v2.csv", "../data/train-jpg", img_size)
X_train, X_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2)

100%|██████████| 40479/40479 [00:35<00:00, 1139.60it/s]


In [4]:
generator = ImageGenerator()
flow = generator.get_train_generator(X_train, y_train)
validation_data = (X_valid, y_valid)

In [ ]:
from tensorflow.contrib.keras.api.keras.callbacks import ModelCheckpoint

filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True)

In [ ]:
model = KerasModel(img_size)
learn_rates = [0.001, 0.0001, 0.00001]
epochs_list = [10, 5, 5]
train_losses, val_losses = [], []

for lr, epochs in zip(learn_rates, epochs_list):
    tmp_train_losses, tmp_val_losses, fbeta_score = model.fit(flow, epochs, lr, (X_valid, y_valid), train_callbacks=[checkpoint])
    train_losses += tmp_train_losses
    val_losses += tmp_val_losses

Epoch 1/10
300/300 [==============================] - 89s - loss: 0.3131 - acc: 0.8790 - val_loss: 0.2646 - val_acc: 0.9047
Epoch 2/10
 49/300 [===>..........................] - ETA: 66s - loss: 0.1719 - acc: 0.9335

In [ ]:
model.load_weights("weights.best.hdf5")
print("Weights loaded")

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend();

In [ ]:
fbeta_score